# Homework 6: Mixed effects

This homework assignment is designed to give you practice fitting and interpreting mixed effects models. 

We will be using the **LexicalData.csv** and **Items.csv** files from the *Homework/lexDat* folder in the class GitHub repository again. 

This data is a subset of the [English Lexicon Project database](https://elexicon.wustl.edu/). It provides the reaction times (in milliseconds) of many subjects as they are presented with letter strings and asked to decide, as quickly and as accurately as possible, whether the letter string is a word or not. The **Items.csv** provides characteristics of the words used, namely frequency (how common is this word?) and length (how many letters?). Unlike in the previous homework, there isn't any missing data in the **LexicalData.csv** file. 

*Data courtesy of Balota, D.A., Yap, M.J., Cortese, M.J., Hutchison, K.A., Kessler, B., Loftis, B., Neely, J.H., Nelson, D.L., Simpson, G.B., & Treiman, R. (2007). The English Lexicon Project. Behavior Research Methods, 39, 445-459.*

---
## 1. Loading and formatting the data (1 point)

Load in data from the **LexicalData.csv** and **Items.csv** files. As in the previous homeworks, remove the commas from the reaction times and convert them from strings to numbers. Use `left_join` to add word characteristics `Length` and `Log_Freq_Hal` from **Items** to **LexicalData**. 

*Note: the `Freq_HAL` variable in **Items.csv** has a similar formatting issue, using string values with commas. We're not going to worry about fixing this since we're only using `Log_Freq_HAL`, which is the natural log transformation of `Freq_HAL`, in this homework.*

In [27]:
# WRITE YOUR CODE HERE
getwd()

[1] "/Users/tim/Documents/GradSchool/Coursework/S22/85-732/nolan_DSPN_S22/Homeworks/HW6"

In [28]:
LexicalData <- read.csv(file = '../lexDat/LexicalData.csv')

LexicalData$D_RT <- as.numeric(gsub(',','',LexicalData$D_RT))
LexicalData <- rename(LexicalData, Word = D_Word)

dim(LexicalData)
head(LexicalData)

[1] 62610     7

,Sub_ID,Trial,Type,D_RT,Word,Outlier,D_Zscore
,<int>,<int>,<int>,<dbl>,<chr>,<chr>,<dbl>
1,157,1,1,710,browse,false,-0.437
2,67,1,1,1094,refrigerant,false,0.825
3,120,1,1,587,gaining,false,-0.645
4,21,1,1,984,cheerless,false,0.025
5,236,1,1,577,pattered,false,-0.763
6,236,2,1,715,conjures,false,-0.364


In [29]:
Items <- read.csv(file = '../lexDat/Items.csv')

keeps <- c("Word","Length","Log_Freq_HAL")
Items <- Items[keeps]

dim(Items)
head(Items)

[1] 30959     3

,Word,Length,Log_Freq_HAL
,<chr>,<int>,<dbl>
1,synergistic,11,5.649
2,synonymous,10,6.858
3,syntactical,11,4.736
4,synthesis,9,8.816
5,synthesized,11,7.904
6,synthesizer,11,7.237


In [30]:
library(tidyverse)

LexicalData <- dplyr::left_join(Items,LexicalData)

dim(LexicalData)
head(LexicalData)

Joining, by = "Word"


[1] 62610     9

,Word,Length,Log_Freq_HAL,Sub_ID,Trial,Type,D_RT,Outlier,D_Zscore
,<chr>,<int>,<dbl>,<int>,<int>,<int>,<dbl>,<chr>,<dbl>
1,synergistic,11,5.649,148,449,1,776,false,0.125
2,synonymous,10,6.858,20,146,1,1525,false,1.235
3,syntactical,11,4.736,162,47,1,1365,false,1.768
4,synthesis,9,8.816,155,93,1,759,false,0.745
5,synthesized,11,7.904,94,49,1,2850,false,4.134
6,synthesizer,11,7.237,272,182,1,1966,false,2.699


---
## 2. Model fitting (4 points)

First, fit a linear model with `Log_Freq_HAL` and `Length` as predictors, and `D_RT` as the output. Include an interaction term. Use `summary()` to look at the model output. 

In [45]:
# WRITE YOUR CODE HERE

lin_model <- lm(D_RT ~ Log_Freq_HAL * Length, data=LexicalData)

Now, install `lme4` using `install.packages()` and then load the library. 

In [46]:
summary(lin_model)


Call:
lm(formula = D_RT ~ Log_Freq_HAL * Length, data = LexicalData)

Residuals:
     Min       1Q   Median       3Q      Max 
-1118.01  -205.23   -86.95    90.77  3147.07 

Coefficients:
                    Estimate Std. Error t value Pr(>|t|)    
(Intercept)         610.1903    14.6678  41.601  < 2e-16 ***
Log_Freq_HAL         -6.0239     1.9678  -3.061  0.00221 ** 
Length               47.7531     1.6368  29.175  < 2e-16 ***
Log_Freq_HAL:Length  -2.9421     0.2348 -12.528  < 2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 359.1 on 62606 degrees of freedom
Multiple R-squared:  0.09473,	Adjusted R-squared:  0.09469 
F-statistic:  2184 on 3 and 62606 DF,  p-value: < 2.2e-16


In [39]:
# WRITE YOUR CODE HERE
install.packages("lme4")

Installing package into ‘/opt/homebrew/lib/R/4.1/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘minqa’, ‘nloptr’, ‘RcppEigen’




In [40]:
library(lme4)

Loading required package: Matrix


Attaching package: ‘Matrix’


The following objects are masked from ‘package:tidyr’:

    expand, pack, unpack




Now fit a mixed effects model that includes the same predictors as the linear model above, as well as random intercepts for `Sub_ID` (i.e., cases where subject ID shifts the RT mean). Use `summary()` to look at the model output. 

In [47]:
# WRITE YOUR CODE HERE

me_model <- lmer(D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID), data=LexicalData)

In [48]:
summary(me_model)

Linear mixed model fit by REML ['lmerMod']
Formula: D_RT ~ Log_Freq_HAL * Length + (1 | Sub_ID)
   Data: LexicalData

REML criterion at convergence: 888235.6

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-4.5058 -0.5472 -0.1568  0.3103 10.7381 

Random effects:
 Groups   Name        Variance Std.Dev.
 Sub_ID   (Intercept) 46333    215.3   
 Residual             82978    288.1   
Number of obs: 62610, groups:  Sub_ID, 299

Fixed effects:
                    Estimate Std. Error t value
(Intercept)         616.8445    17.1522  35.963
Log_Freq_HAL         -7.4374     1.5830  -4.698
Length               47.7477     1.3162  36.277
Log_Freq_HAL:Length  -2.8778     0.1888 -15.239

Correlation of Fixed Effects:
            (Intr) Lg_F_HAL Length
Log_Frq_HAL -0.645                
Length      -0.656  0.917         
Lg_Fr_HAL:L  0.582 -0.942   -0.923

---
## 3. Model assessment (4 points)

Compare the three t-values for the fixed effects and the mixed effects models. How do they differ, and why? 

For ease of comparison, here's a table of the t-values in question:

| Coefficient         | t val, Linear Model | t val, Mixed Effect Model|
| :---                |      ---:           |          ---:            |
| (Intercept)         | 41.601              | 35.963                   |
| Log_Freq_HAL        | -3.061              | -4.698                   |
| Length              | 29.175              | 36.277                   |
| Log_Freq_HAL:Length | -12.528             | -15.239                  |

Let "\_LM" denote "Linear Model," and "\_ME" denote "Mixed Effect Model."

First, for the intercept term, $t_{LM} > t_{ME}$, though both are very large. The null hypothesis for both models is $H_0 = 0$, which we reject in both cases.

For all three fixed effects, $|t_{LM}| < |t_{ME}|$. Our t-value is further on the tails of the null distribution, and the corresponding mixed effects model yields a  regression with higher confidence across all of its fixed effects due to the introduction of the random subject ID effect correction. That is, the fit of the model has improved.

Use the Aikeke Information Criterion (AIC) to compare these two models. Which one is better? 

In [44]:
# WRITE YOUR CODE HERE
AIC(lin_model)
AIC(me_model)

[1] 914436.4

[1] 888247.6

The lower the AIC, the better. Thus, just as we came to the conclusion by looking at the test statistics above (i.e., classical Fisherian statistics framework), the mixed effects model is better than the pure linear model according to information theoretic fraework, too.

---
##  4. Reflection (1 point)

What other random effects could be controlled for in this data set? 

Assume that globally, there is no temporal change to the reaction time response as the experiment runs its course. That is, there's no habituation, priming, fatigue, and the subject maintains consistent effort throughout (whether or not these assumptions are reasonable are task-dependent). If the assumption were to hold true, you could also correct for Trial Number. None of the other columns of LexicalData.csv stand out to me as potential random effects.

**DUE:** 5pm EST, March 25, 2021

**IMPORTANT** Did you collaborate with anyone on this assignment? If so, list their names here. 
> *Someone's Name*